In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

import pandas as pd
from collections import OrderedDict

import pprint
import re
import numpy as np
import matplotlib.pyplot as plt
import random

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
init_notebook_mode(connected=True)  
import glob
import uproot 
import ROOT

import platform
print('Python:', platform.python_version())

Welcome to JupyROOT 6.26/06
Python: 3.10.8


In [2]:
# run = 10645
run = 10647

filenames = glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc*_pedestal-*{run}-0000.prdf.root")
filenames.sort()
pprint.pprint(filenames)

['/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc00_pedestal-00010647-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc01_pedestal-00010647-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc02_pedestal-00010647-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc03_pedestal-00010647-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc04_pedestal-00010647-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc05_pedestal-00010647-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc06_pedestal-00010647-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc07_pedestal-00010647-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc08_pedestal-00010647-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_eb

In [3]:
files = dict()
for filename in filenames: 
    file = ROOT.TFile.Open(filename)
    sector = re.match(".*ebdc([0-9][0-9])_.*", filename).group(1)
    files[sector] = file


pprint.pprint(files);

{'00': <cppyy.gbl.TFile object at 0x9036820>,
 '01': <cppyy.gbl.TFile object at 0x92ef380>,
 '02': <cppyy.gbl.TFile object at 0x928d600>,
 '03': <cppyy.gbl.TFile object at 0x9285990>,
 '04': <cppyy.gbl.TFile object at 0x92e1630>,
 '05': <cppyy.gbl.TFile object at 0x9277ae0>,
 '06': <cppyy.gbl.TFile object at 0x927e090>,
 '07': <cppyy.gbl.TFile object at 0x9280c40>,
 '08': <cppyy.gbl.TFile object at 0x938df70>,
 '09': <cppyy.gbl.TFile object at 0x9390b10>,
 '10': <cppyy.gbl.TFile object at 0x939e6a0>,
 '11': <cppyy.gbl.TFile object at 0x93a1660>,
 '12': <cppyy.gbl.TFile object at 0x93a4140>,
 '13': <cppyy.gbl.TFile object at 0x93eaa00>,
 '14': <cppyy.gbl.TFile object at 0x93ed470>,
 '15': <cppyy.gbl.TFile object at 0x93f0090>,
 '16': <cppyy.gbl.TFile object at 0x93f2c30>,
 '18': <cppyy.gbl.TFile object at 0x9405880>,
 '19': <cppyy.gbl.TFile object at 0x9408320>,
 '20': <cppyy.gbl.TFile object at 0x940afc0>,
 '21': <cppyy.gbl.TFile object at 0x940dae0>,
 '22': <cppyy.gbl.TFile object at 

In [10]:
files['02'].ls()
files['02'].PacketTree.Show(0)
files['02'].SampleTree.Show(0)
files['02'].TaggerTree.Show(0)

TFile**		/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc02_pedestal-00010647-0000.prdf.root	
 TFile*		/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc02_pedestal-00010647-0000.prdf.root	
  OBJ: TTree	SampleTree	Each entry is one waveform : 0 at: 0xa4ab230
  KEY: TTree	PacketTree;1	Each entry is one packet
  KEY: TTree	SampleTree;1	Each entry is one waveform
  KEY: TTree	TaggerTree;1	Each entry is one tagger for level 1 trigger or endat tag
======> EVENT:0
 packet          = 4020
 frame           = 2
 nWaveormInFrame = 637
 nTaggerInFrame  = 2
 maxFEECount     = 26
======> EVENT:0
 packet          = 4020
 frame           = 2
 nWaveormInFrame = 637
 maxFEECount     = 26
 nSamples        = 360
 adcSamples      = 76, 
                  74, 79, 78, 74, 80, 78, 74, 72, 80, 82, 
                  75, 74, 80, 75, 75, 78, 78, 74, 76
 fee             = 22
 sampaAddress    = 4
 sampaChannel    = 1
 Channel         = 129
 BCO             = 564491
 checksum        =

In [5]:
%%cpp -d

#include<sPhenixStyle.C>

In [6]:
%%cpp

SetsPhenixStyle()

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [7]:
%jsroot on

In [8]:
c1 = ROOT.TCanvas( 'checksumErrorQA', 'checksumErrorQA', 1500, 1200)

c1.Divide(4,3)

i = 1
for sector, file in files.items():
    
    c1.cd(i)
    i=i+1
    
    hAllWaveForm = ROOT.TH1F(f'hAllWaveForm{sector}', ';fee*8+sampaAddress [0..207]', 26*8, -.5, 26*8 -.5);
    hErrorWaveForm = ROOT.TH1F(f'hErrorWaveForm{sector}', f';Sector{sector}: fee*8+sampaAddress [0..207];Probability of CheckSum Error', 26*8, -.5, 26*8 -.5);

    files[sector].SampleTree.Draw(f"fee*8+sampaAddress >> hAllWaveForm{sector}","","goff");
    files[sector].SampleTree.Draw(f"fee*8+sampaAddress >> hErrorWaveForm{sector}","checksumError>0","goff");
    
    hErrorWaveForm.Divide (hErrorWaveForm, hAllWaveForm, 1, 1, 'B');
    hErrorWaveForm.DrawClone();

c1.Draw()

In [9]:

c1 = ROOT.TCanvas( 'ADCQA', 'ADCQA', 1500, 1200)

c1.Divide(4,3)

i = 1
for sector, file in files.items():
    

    pad = c1.cd(i)
    i=i+1

    pad.SetLogz()
    
    hADC = ROOT.TH2F(f'hADC{sector}', f';Sector{sector}: ADC Time bin [1/20MHz]; ADC [ADU]', 360, 0, 360, 256,0,1024);

    files[sector].SampleTree.Draw(f"adcSamples:Iteration$>>hADC{sector}","checksumError==0","colz");
    
    hADC.DrawClone("colz");
    
c1.Draw()